In [0]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import tweepy
import GetOldTweets3 as got

In [0]:
#Twitter credentials for the app
consumer_key = 'XXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_key= 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [0]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [0]:
AirAsia = "./AirAsia_Data.csv"

#### upload tweets

In [0]:
#api.update_status("Test with python twitter API")

#### extract tweets

In [0]:
# Collect tweets of compliants for Asia
date_since = "2020-03-04"
date_until = "2020-03-06"

In [0]:
search_words = "@AirAsia"
tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since,
              until=date_until).items()
compliant_tweets = {}
for tweet in tweets:
    tweet_id = str(tweet.id)
    #auth_id_user_id = tweet.user.id
    auth_id_screen_name = tweet.user.screen_name
    tweet_text = tweet.text
    tweet_time = str(tweet.created_at)
    # add to the dictionary
    compliant_tweets[tweet_id] = {}
    compliant_tweets[tweet_id]['created_at'] = tweet_time
    compliant_tweets[tweet_id]['user_id'] = auth_id_screen_name
    compliant_tweets[tweet_id]['text'] = tweet_text

TweepError: Twitter error response: status code = 429

In [0]:
# Collect tweets of reply from AirAsia
username = 'AirAsia'
AirAsia_tweets = {}
for tweet_id in compliant_tweets:
    for airaisa_tweet in api.user_timeline(id = username,since_id = tweet_id):
        if airaisa_tweet.in_reply_to_screen_name == compliant_tweets[tweet_id]['user_id']:
            airasia_tweet_id = str(airaisa_tweet.id)
            airasia_auth_id_screen_name = airaisa_tweet.user.screen_name
            airasia_tweet_text = airaisa_tweet.text
            airasia_tweet_time = str(airaisa_tweet.created_at)
            compliant_tweets[tweet_id]['response_tweet_id'] = airasia_tweet_id
            AirAsia_tweets[airasia_tweet_id] = {}
            AirAsia_tweets[airasia_tweet_id]['created_at'] = airasia_tweet_time
            AirAsia_tweets[airasia_tweet_id]['user_id'] = airasia_auth_id_screen_name
            AirAsia_tweets[airasia_tweet_id]['text'] = airasia_tweet_text
            AirAsia_tweets[airasia_tweet_id]['in_response_to_tweet_id'] = tweet_id
            break

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [0]:
# Deal with the tweets of another account : AirAsiaSupport
search_words = "@AirAsiaSupport"
tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since,
              until=date_until).items()
for tweet in tweets:
    tweet_id = str(tweet.id)
    auth_id_screen_name = tweet.user.screen_name
    tweet_text = tweet.text
    tweet_time = tweet.created_at
    # add to the dictionary
    if tweet_id not in compliant_tweets:
        compliant_tweets[tweet_id] = {}
        compliant_tweets[tweet_id]['created_at'] = tweet_time
        compliant_tweets[tweet_id]['user_id'] = auth_id_screen_name
        compliant_tweets[tweet_id]['text'] = tweet_text

In [0]:
# Collect tweets of reply from AirAsiaSupport
username = 'AirAsiaSupport'
for tweet_id in compliant_tweets:
    for airaisa_tweet in api.user_timeline(id = username,since_id = tweet_id):
        if airaisa_tweet.in_reply_to_screen_name == compliant_tweets[tweet_id]['user_id']:
            airasia_tweet_id = str(airaisa_tweet.id)
            airasia_auth_id_screen_name = airaisa_tweet.user.screen_name
            airasia_tweet_text = airaisa_tweet.text
            airasia_tweet_time = airaisa_tweet.created_at
            compliant_tweets[tweet_id]['response_tweet_id'] = airasia_tweet_id
            AirAsia_tweets[airasia_tweet_id] = {}
            AirAsia_tweets[airasia_tweet_id]['created_at'] = airasia_tweet_time
            AirAsia_tweets[airasia_tweet_id]['user_id'] = airasia_auth_id_screen_name
            AirAsia_tweets[airasia_tweet_id]['text'] = airasia_tweet_text
            AirAsia_tweets[airasia_tweet_id]['in_response_to_tweet_id'] = tweet_id
            break

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [0]:
with open('./compliants.json', 'w') as json_file:
    json.dump(compliant_tweets, json_file)
with open('./AirAsia.json', 'w') as json_file:
    json.dump(AirAsia_tweets, json_file)
